In [2]:
from langchain import OpenAI, VectorDBQA, LLMChain
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [3]:
import sys
sys.path.append(r"C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding")
from utils import ChromaDBManager, LangChainAI, QDrantDBManager, EmbeddingFunction

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [5]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
embedding = EmbeddingFunction('openAI').embedder
vectore_store=qdrantClient = QDrantDBManager(
    url="http://ec2-18-209-145-26.compute-1.amazonaws.com:6333/dashboard",
    port=6333,
    collection_name="rio-rag-platform",
    vector_size=1536,
    embedding=embedding,
    record_manager_url=r"sqlite:///C:\Users\ELAFACRB1\Codice\GitHub\media-chat-service\src\embedding\record_manager_cache.sql"
)
vectore_store_client=vectore_store.vector_store

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Collection rio-rag-platform already exists!


In [6]:
COLLECTION_NAME="rio-rag-platform"

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [8]:
query='what the role on product owner in Safe? '

In [9]:
qa_chain(query)

c:\Users\ELAFACRB1\venvs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'what the role on product owner in Safe? ',
 'result': 'The role of the Product Owner in SAFe (Scaled Agile Framework) is responsible for connecting with the customer, contributing to the vision and roadmap, managing and prioritizing the team backlog, supporting the team in delivering value, and getting and applying fast feedback. The Product Owner works closely with the Agile Team to ensure that the product or solution being developed meets the needs of the customer and aligns with the overall vision and goals of the organization.',
 'source_documents': [Document(page_content='efficiencies over timeFOR REFERENCE ONLY 2-13 Workbook 38 © Scaled Agile, Inc. For personal use only - Rosario Laface - Nov 16, 2023 (ID:0056T00000949fqQAA)  © Scaled Agile. Inc. © Scaled Agile, Inc.  2.2 The Scrum Master / Team Coach and  Product Owner roles 2-16 © Scaled Agile. Inc. Team Charter Agile Purpose Success Measures Team Members Team Type & ResponsibilitiesDefinition of Done Distinctive Com

Parent document

In [10]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [15]:
from langchain_community.vectorstores import Qdrant

In [16]:
Qdrant

langchain_community.vectorstores.qdrant.Qdrant

In [11]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [12]:
store = InMemoryStore()

In [18]:
retriever = ParentDocumentRetriever(
    vectorstore=vectore_store.vector_store,
    docstore=store,
    child_splitter=child_splitter,
)

In [20]:
from langchain.document_loaders import RSSFeedLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
urls = ["https://medium.com/feed/tag/quantum-computing"]
loader = RSSFeedLoader(urls=urls)
data = loader.load()

In [21]:
retriever.add_documents(data, ids=None)

In [24]:
retriever

ParentDocumentRetriever(vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x000001AD239F8C90>, docstore=<langchain.storage.in_memory.InMemoryBaseStore object at 0x000001AD247C1E50>, child_splitter=<langchain.text_splitter.RecursiveCharacterTextSplitter object at 0x000001AD251CFF50>)

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectore_store_client.as_retriever(),
    return_source_documents=True,
)

In [23]:
qa_chain('whats the most recent computer for quantum computing?')

{'query': 'whats the most recent computer for quantum computing?',
 'result': "The most recent computer for quantum computing is the Sycamore processor developed by Google. It made headlines in 2019 when Google claimed to have achieved quantum supremacy with this processor. However, it's important to note that the field of quantum computing is rapidly evolving, and new advancements are being made by various companies and research institutions.",
 'source_documents': [Document(page_content='The Future of Quantum Computing', metadata={'authors': [], 'description': 'Are you tired of waiting ages for data processing to complete? Frustrated by the limitations of conventional computing power? If so, you’re not alone. In a world where data is king, speed and…', 'doc_id': '7ed35252-4320-4062-a7e2-aada649aa0b1', 'feed': 'https://medium.com/feed/tag/quantum-computing', 'language': 'en', 'link': 'https://medium.com/@krupalu.datta/quantum-supremacy-redefining-speed-and-efficiency-in-data-processin